In [1]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

In [2]:
import pandas as pd, numpy as np, seaborn as sns
import matplotlib.pyplot as plt
import wandb
from datasets import load_metric, Dataset
from IPython.display import Markdown, display
from IPython.core.debugger import set_trace
import plotly.express as px

from datasets import load_dataset, Dataset

from lexicalrichness import LexicalRichness
import functools
import string
import psutil
from collections import defaultdict
from spacy_wordnet.wordnet_annotator import WordnetAnnotator 
import operator
import spacy
import textstat
import difflib as dl
from itertools import groupby
from operator import itemgetter

In [3]:
from analysis_functions import *

In [151]:
seed = 420
np.random.seed(seed)
run_name = "mild-jazz-178"
run_id = "36206pkc"
wandb_run_path = f"uts_nlp/travis_attack/{run_id}"  # get this from wandb overview section 
pd.options.mode.chained_assignment = None  # default='warn'


# Init W&B run object
api = wandb.Api()
run = api.run(wandb_run_path)
config = run.config

# Load CSV files
path_results = f"../model_checkpoints/travis_attack/{run_name}/"
train            = pd.read_csv(path_results + "train.csv")
valid            = pd.read_csv(path_results + "valid.csv")
test             = pd.read_csv(path_results + "test.csv")
training_step    = pd.read_csv(path_results + "training_step.csv")
training_summary = pd.read_csv(path_results + "training_summary.csv")

## Dataset-level stats

In [152]:
df_training_step = postprocess_df(training_step, filter_idx=None)
df_train = postprocess_df(train, filter_idx=None)
df_valid = postprocess_df(valid, filter_idx=None)


#### Calculating text statistics for the original examples. ####


#### Calculating text statistics for paraphrases. ####


#### Calculating text pair statistics ####


#### Calculating text statistics for the original examples. ####


#### Calculating text statistics for paraphrases. ####


#### Calculating text pair statistics ####


#### Calculating text statistics for the original examples. ####


#### Calculating text statistics for paraphrases. ####


#### Calculating text pair statistics ####



In [229]:
#df_training_step.to_pickle(path_results + "df_temp_training_step.pkl")
#df_train.to_pickle(path_results + "df_temp_train.pkl")
#df_valid.to_pickle(path_results + "df_temp_valid.pkl")

df_training_step = pd.read_pickle(path_results + "df_temp_training_step.pkl")
df_train = pd.read_pickle(path_results + "df_temp_train.pkl")
df_valid = pd.read_pickle(path_results + "df_temp_valid.pkl")

In [230]:
df_training_step['data_split'] = 'training_step'
df_train['data_split'] = 'eval_train'
df_valid['data_split'] = 'eval_valid'
df_concat = pd.concat([df_training_step,df_train,df_valid])
df_concat = df_concat.reset_index(drop=True)
# Otherwise we get a big spike at 0
df_concat.loc[df_concat.epoch_of_first_label_flip == 0, 'epoch_of_first_label_flip'] = None
fig_l = []

In [ ]:
hist_config_dicts = [
    {
        'colname': 'epoch_of_first_label_flip', 
        'xlabel': "Epoch of first label flip", 
        'desc': "Cumulative prob Epoch of first label flip for each original example",
        'cumulative': True,
    },
    {
        'colname': 'idx_n_unique_pp', 
        'xlabel': "Unique paraphrases per original example", 
        "desc": "Number of generated unique paraphrases per original example during training", 
        'cumulative': False,
    },
    {
        'colname': 'idx_n_pp_changes', 
        'xlabel': "Paraphrase changes per original example", 
        "desc": "Number of paraphrase changes per original example during training", 
        'cumulative': False,
    },
]


fig_l = []
line_colnames = [o for o in df_concat.columns if "_diff" in o] + [
    'rouge_score', "is_truncation", 'any_phrase_capitalised', 'any_phrase_decapitalised', 
'n_segments_inserted', 'n_segments_removed', 'n_tokens_inserted', 'n_tokens_removed','edit_distance_token_level']
for colname in line_colnames: 
    fig = plot_epoch_line_charts(df_concat, colname)
    fig_l.append({f"pp_metrics/{colname}":fig })
    
for d in hist_config_dicts: 
    fig = plot_idx_hist(df_concat, d['colname'],d['xlabel'],d['cumulative'])
    fig_l.append({f"pp_metrics/{d['colname']}":fig })

d1 = {k: v for d in fig_l for k, v in d.items()}
#for fig in d1.values(): fig.show()
run1 = wandb.init(job_type="analysis")
run1.log(d1)
run1.finish()

In [7]:
#display_all(df_concat[['orig_l', 'pp_l','removals','insertions','unchanged',  'is_truncation']].sample(10))

,orig_l,pp_l,removals,insertions,unchanged,is_truncation
33075,this riveting world war ii moral suspense story deals with the shadow side of american culture : racial prejudice in its ugly and diverse forms .,This riveting world war ii moral suspense story deals with the shadow side of american culture : racial prejudice in its ugly and diverse forms.,['this'],['This'],['riveting world war ii moral suspense story deals with the shadow side of american culture: racial prejudice in its ugly and diverse forms.'],False
119115,the basic premise is intriguing but quickly becomes distasteful and downright creepy .,The basic premise is intriguing but quickly becomes distasteful and downright creepy.,['the'],['The'],['basic premise is intriguing but quickly becomes distasteful and downright creepy.'],False
66312,"a time machine , a journey back to your childhood , when cares melted away in the dark theater , and films had the ability to mesmerize , astonish and entertain .","A time machine, a journey back to your childhood, when cares melted away in the dark theater, and films had the ability to mesmerize, astonish and entertain.",['a'],['A'],"['time machine, a journey back to your childhood, when cares melted away in the dark theater, and films had the ability to mesmerize, astonish and entertain.']",False
1089,"'compleja e intelectualmente retadora , el ladrón de orquídeas es uno de esos filmes que vale la pena ver precisamente por su originalidad . '","'compleja e intelectualmente retadora, el ladrón de orquídeas es uno de esos filmes that vale la pena ver precisamente por su originalidad.","['que', ""'""]",['that'],"[""' compleja e intelectualmente retadora, el ladrón de orquídeas es uno de esos filmes"", 'vale la pena ver precisamente por su originalidad.']",True
197922,"sadly , 'garth' hasn't progressed as nicely as 'wayne . '","sadly, 'garth' hasn't progressed as nicely as 'wayne.","[""'""]",[],"[""sadly,' garth' has n't progressed as nicely as' wayne.""]",True
165106,remember when bond had more glamour than clamor ? no more .,"remember when bond had more glamour than clamor, no more.",['?'],"[',']","['remember when bond had more glamour than clamor', 'no more.']",False
70484,a wonderfully speculative character study that made up for its rather slow beginning by drawing me into the picture .,A wonderfully speculative character study that made up for its rather slow beginning by drawing me into the picture.,['a'],['A'],['wonderfully speculative character study that made up for its rather slow beginning by drawing me into the picture.'],False
197723,"the origin story is well told , and the characters will not disappoint anyone who values the original comic books . it's in the action scenes that things fall apart .","The origin story is well told, and the characters will not disappoint anyone who values the original comic books.","['the', ""it 's in the action scenes that things fall apart.""]",['The'],"['origin story is well told, and the characters will not disappoint anyone who values the original comic books.']",True
152976,"despite all evidence to the contrary , this clunker has somehow managed to pose as an actual feature movie , the kind that charges full admission and gets hyped on tv and purports to amuse small children and ostensible adults .","Despite all evidence to the contrary, this clunker has somehow managed to pose as an actual feature movie, the kind that charges full admission and gets hyped on tv and purports to amuse small children and ostensible adults.",['despite'],['Despite'],"['all evidence to the contrary, this clunker has somehow managed to pose as an actual feature movie, the kind that charges full admission and gets hyped on tv and purports to amuse small children and ostensible adults.']",False
73167,can be viewed as pure composition and form -- film as music,can be viewed as pure composition and form -- film as music.,[],['.'],['can be viewed as pure composition and form -- film as music'],False
